<a href="https://colab.research.google.com/github/0x6d61/ipynbstorage/blob/master/log_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
!wget https://raw.githubusercontent.com/Morzeux/HttpParamsDataset/master/payload_full.csv


--2021-10-16 23:50:27--  https://raw.githubusercontent.com/Morzeux/HttpParamsDataset/master/payload_full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1996540 (1.9M) [text/plain]
Saving to: ‘payload_full.csv.28’

payload_full.csv.28 100%[===================>]   1.90M  --.-KB/s    in 0.04s   

2021-10-16 23:50:27 (48.5 MB/s) - ‘payload_full.csv.28’ saved [1996540/1996540]



In [142]:
import pandas as pd
import numpy as np
import re
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV

classifier =  RandomForestClassifier(random_state=0,criterion="gini",max_depth=11,n_estimators=500)
vectorizer = DictVectorizer(sparse=False)

csv = pd.read_csv('payload_full.csv',sep=",")
X = csv.payload.values
y = [0 if v == "norm" else 1 for v in csv.label.values] 

In [143]:
def symbol_analysis(log):
  lst = list(log)
  data = dict()
  data["less_than_sign"] = len([v for v in lst if v == "<"])
  data["left_parenthesis"] = len([v for v in lst if v == "("])
  data["space"] = len([v for v in lst if v == " " ])
  #data["solidus"] = len([v for v in lst if v == "/"])
  data["semicolon"] = len([v for v in lst if v == ";" ])
  data["full_stop"] = len([v for v in lst if v == "."])
  data["quotation_mark"] = len([v for v in lst if v == '"'])
  data["apostrophe"] = len([v for v in lst if v == "'"])
  data["question_mark"] = len([v for v in lst if v == "?"])
  data["number_sign"] = len([v for v in lst if v == "#"])
  data["percent_sign"] = len([v for v in lst if v == '%'])
  data["symbol_num"] =  sum([i for i in data.values()])
  data["double_full_stop"] = len(re.findall('(\.\.)+',log))
  return data

In [144]:
X = [symbol_analysis(str(x)) for x in X]
vectorizer.fit(X)
X = vectorizer.transform(X)
mmscaler = MinMaxScaler(feature_range=(0, 1), copy=True)
mmscaler.fit(X)
X = mmscaler.transform(X)
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True) 
classifier.fit(X_train,y_train)
#scores = classifier.predict_proba(X_test)[:,1]
print(classifier.score(X_train,y_train))
print(classifier.score(X_test,y_test))

0.9882403433476394
0.9877687652890433


In [145]:
classifier.predict_proba(vectorizer.transform([symbol_analysis('/passwd')]))[:,1]

array([0.00628121])